In [1]:
shift_param = 1

In [2]:
import pandas as pd
import numpy as np

from pathlib import Path
import os
import datetime as dt
import pickle

from src.utils import fetch_latest_fpl_data

from sklearn.linear_model import LogisticRegression
from scipy.stats import poisson

import matplotlib.pyplot as plt
import plotly.express as px

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)

In [3]:
# model for estimating bonus points based on gameweek bps
model_path = Path(f"../models/logistic_regression_for_bonus_points.pkl")
with open(model_path, "rb") as f:
    clf = pickle.load(f)

/Users/jaakkotoivonen/Documents/my_stuff/Fantasy-Premier-League/venv23-24/lib/python3.10/site-packages/sklearn/base.py:347: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.0.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


# Functions

In [4]:
def fpl_data_processing(df, columns):

    xg_data = []
    xa_data = []
    xga_data = []
    for ix, row in df.iterrows():
        my_gameweek = row['gameweek']
        xg_data.append( row[f'xG_week{my_gameweek}'] )
        xa_data.append( row[f'xA_week{my_gameweek}'] )
        xga_data.append( row[f'xGA_week{my_gameweek}'] )

    df['gameweek_xG'] = xg_data
    df['gameweek_xA'] = xa_data
    df['gameweek_xGA'] = xga_data

    df_new = df[columns].copy()

    return df_new

In [5]:
def my_fill_na(x, gameweek_col, diff_col):
    '''Fill nan values for first items for grouped variables where diff is calculated. But also don't fill for season 22-23,
    where data is missing for a number of weeks at the beginning of the season.'''
    my_value = x[diff_col] if (np.isnan(x[gameweek_col])) & (x['minutes']<=90) else x[gameweek_col]
    return my_value
    

In [6]:
def calculate_xPoints(x,clf):
    """Expected points for a given gameweek given underlying stats for that gameweek."""

    clean_sheet_points = np.array([4,4,1,0])
    goal_points = np.array([6,6,5,4])

    # calculate expexted points
    points_played = np.array([1 if x['gameweek_minutes']>0 else 0])
    points_played_over_60 = np.array([1 if x['gameweek_minutes']>=60 else 0])
    points_xG = goal_points[x['element_type']-1] * x['gameweek_xG']
    points_xA = x['gameweek_xA'] * 3
    clean_sheet_probability = np.array(poisson.pmf(0,x['team_xGA']))
    points_clean_sheet = [clean_sheet_points[x['element_type']-1] * clean_sheet_probability if x['gameweek_minutes']>=60 else 0]
    points_saves = x['gameweek_saves'] // 3
    points_penalty_saves = x['gameweek_penalties_saved'] * 5 * 0.21 #points for save times approx. probability of penalty save
    #penalty_for_penalty_miss = x['Performance_PKatt'] * (-2*0.21) # this data only on fbref
    # estimate bonus points
    if not np.isnan(x['gameweek_bps']):
        y_pred_prob = clf.predict_proba(np.array(x['gameweek_bps']).reshape(-1, 1))
    else:
        # return nan if bonus points can't be estimated 
        return np.nan
    points_bonus = np.matmul(y_pred_prob, np.array([0,1,2,3]).reshape((4,1)))
    
    # penalty for possible points deductions based on goals conceded
    xGA = x['team_xGA']
    # calculate penalty
    xGA_conceded_penalty = -(poisson.pmf(2,xGA)+poisson.pmf(3,xGA))-(poisson.pmf(4,xGA)+poisson.pmf(5,xGA))-(poisson.pmf(6,xGA)+poisson.pmf(7,xGA))-(poisson.pmf(8,xGA)+poisson.pmf(9,xGA)-(poisson.pmf(10,xGA)+poisson.pmf(11,xGA)))
    # apply penalty only to GK and DEF
    if (x['element_type']==1) | (x['element_type']==2):
        xGA_conceded_penalty = xGA_conceded_penalty
    else:
        xGA_conceded_penalty = 0
    # scale penalty with playing time
    xGA_conceded_penalty = (x['gameweek_minutes'] / 90) * xGA_conceded_penalty

    penalty_for_cards = [-3 if x['gameweek_red_cards']==1 else -1 if x['gameweek_yellow_cards']==1 else 0]
    penalty_for_own_goal = -2 * x['gameweek_own_goals']

    # add up all point components
    total_points = float(points_played + points_played_over_60 + points_xG + points_xA + points_clean_sheet + points_saves +\
                    points_penalty_saves + points_bonus + xGA_conceded_penalty +\
                    penalty_for_cards + penalty_for_own_goal)
    
    return total_points

# Fetch data

In [7]:
# fpl data from previous seasons
filepath = Path('../data/modeling/fpl_df.csv')
fpl_df = pd.read_csv(filepath, index_col=0)
display(fpl_df.head())
display(fpl_df.shape)

,assists,bonus,bps,clean_sheets,corners_and_indirect_freekicks_order,creativity,creativity_rank,creativity_rank_type,direct_freekicks_order,dreamteam_count,element_type,event_points,first_name,goals_conceded,goals_scored,ict_index,ict_index_rank,ict_index_rank_type,influence,influence_rank,influence_rank_type,minutes,now_cost,own_goals,penalties_missed,penalties_order,penalties_saved,points_per_game,red_cards,saves,second_name,selected_by_percent,threat,threat_rank,threat_rank_type,total_points,web_name,yellow_cards,team_name,gameweek,season,gameweek_xG,gameweek_xA,gameweek_xGA,gameweek_minutes,team_xG,team_xGA,team_xG_ewm_5,team_xG_ewm_10,team_xG_ewm_20,team_xG_ewm_40,team_xGA_ewm_5,team_xGA_ewm_10,team_xGA_ewm_20,team_xGA_ewm_40,opponent_xG,opponent_xGA,opponent_xG_ewm_5,opponent_xG_ewm_10,opponent_xG_ewm_20,opponent_xG_ewm_40,opponent_xGA_ewm_5,opponent_xGA_ewm_10,opponent_xGA_ewm_20,opponent_xGA_ewm_40,home,gameweek_assists,gameweek_bps,gameweek_creativity,gameweek_goals_scored,gameweek_goals_conceded,gameweek_own_goals,gameweek_penalties_saved,gameweek_red_cards,gameweek_saves,gameweek_threat,gameweek_yellow_cards,gameweek_xPoints,gameweek_assists_ewm_5,gameweek_bps_ewm_5,gameweek_creativity_ewm_5,event_points_ewm_5,gameweek_goals_scored_ewm_5,gameweek_goals_conceded_ewm_5,gameweek_saves_ewm_5,gameweek_threat_ewm_5,gameweek_xG_ewm_5,gameweek_xA_ewm_5,gameweek_xGA_ewm_5,gameweek_minutes_ewm_5,gameweek_xPoints_ewm_5,gameweek_assists_ewm_10,gameweek_bps_ewm_10,gameweek_creativity_ewm_10,event_points_ewm_10,gameweek_goals_scored_ewm_10,gameweek_goals_conceded_ewm_10,gameweek_saves_ewm_10,gameweek_threat_ewm_10,gameweek_xG_ewm_10,gameweek_xA_ewm_10,gameweek_xGA_ewm_10,gameweek_minutes_ewm_10,gameweek_xPoints_ewm_10,gameweek_assists_ewm_20,gameweek_bps_ewm_20,gameweek_creativity_ewm_20,event_points_ewm_20,gameweek_goals_scored_ewm_20,gameweek_goals_conceded_ewm_20,gameweek_saves_ewm_20,gameweek_threat_ewm_20,gameweek_xG_ewm_20,gameweek_xA_ewm_20,gameweek_xGA_ewm_20,gameweek_minutes_ewm_20,gameweek_xPoints_ewm_20,gameweek_assists_ewm_40,gameweek_bps_ewm_40,gameweek_creativity_ewm_40,event_points_ewm_40,gameweek_goals_scored_ewm_40,gameweek_goals_conceded_ewm_40,gameweek_saves_ewm_40,gameweek_threat_ewm_40,gameweek_xG_ewm_40,gameweek_xA_ewm_40,gameweek_xGA_ewm_40,gameweek_minutes_ewm_40,gameweek_xPoints_ewm_40,gameweek_assists_expanding,gameweek_bps_expanding,gameweek_creativity_expanding,event_points_expanding,gameweek_goals_scored_expanding,gameweek_goals_conceded_expanding,gameweek_saves_expanding,gameweek_threat_expanding,gameweek_xG_expanding,gameweek_xA_expanding,gameweek_xGA_expanding,gameweek_minutes_expanding,gameweek_xPoints_expanding,gameweek_assists_expanding_per90,gameweek_bps_expanding_per90,gameweek_creativity_expanding_per90,event_points_expanding_per90,gameweek_goals_scored_expanding_per90,gameweek_goals_conceded_expanding_per90,gameweek_saves_expanding_per90,gameweek_threat_expanding_per90,gameweek_xG_expanding_per90,gameweek_xA_expanding_per90,gameweek_xGA_expanding_per90,gameweek_minutes_expanding_per90,gameweek_xPoints_expanding_per90,xG_overperformance
0,0,0,3,0,NaN,0.0,493,188,4.0,0,2,1,David,0,0,0.0,497,188,0.0,490,188,1,55,0,0,NaN,0,1.0,0,0,Luiz Moreira Marinho,0.9,0.0,479,186,1,David Luiz,0,Arsenal,2,20-21,0.0,0.0,1.9,1.0,1.4,1.9,1.8,1.8,1.8,1.8,0.2,0.2,0.2,0.2,1.9,1.4,1.1,1.1,1.1,1.1,1.5,1.5,1.5,1.5,1.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.993836,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.9,1.0,0.993836,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.9,1.0,0.993836,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.9,1.0,0.993836,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.9,1.0,0.993836,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.9,1.0,0.993836,0.0,270.0,0.0,90.000000,0.0,0.0,0.0,0.0,0.000000,0.0,171.00,90.0,89.445204,NaN
1,1,0,39,1,NaN,51.7,19,15,6.0,0,3,5,Pierre-Emerick,1,1,18.5,13,9,64.6,31,15,180,119,0,0,1.0,0,6.0,0,0,Aubameyang,32.9,69.0,16,6,12,Aubameyang,1,Arsenal,2,20-21,0.1,0.5,1.9,90.0,1.4,1.9,1.8,1.8,1

(21223, 157)

In [8]:
# fpl data from this season
fetch_latest_fpl_data()

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,id,in_dreamteam,news,news_added,now_cost,photo,points_per_game,second_name,selected_by_percent,special,squad_number,status,team,team_code,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text,expected_goals_per_90,saves_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90,team_name,name,gameweek,season,data_retrieved_datetime
0,NaN,NaN,226597,0,0,0,0,0,2,2.8,2.8,1,Gabriel,1.0,5,False,NaN,NaN,50,226597.jpg,1.0,dos Santos Magalhães,31.6,False,NaN,a,1,3,1,26550,26550,115952,115952,0.2,0.2,Gabriel,4,0,0,0,0,0,0,0,0,0,0,0,2,0.2,0.0,0.0,0.0,0,0.00,0.00,0.00,0.02,196,72,554,210,546,209,559,210,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.0,0.00,0.00,0.45,0.00,255,31,218,77,218,77,7,3,0.00,0.00,Arsenal,Gabriel dos Santos Magalhães,1,23-24,2023-08-14 21:41:02.445217
1,NaN,NaN,219847,0,0,0,0,0,3,3.1,3.1,2,Kai,2.0,6,False,NaN,NaN,75,219847.jpg,2.0,Havertz,4.8,False,NaN,a,1,3,2,4273,4273,25704,25704,0.3,0.3,Havertz,90,0,0,0,1,0,0,0,0,0,0,0,11,9.4,30.3,15.0,5.5,1,0.05,0.24,0.29,1.18,130,51,22,16,58,34,54,29,NaN,NaN,NaN,NaN,NaN,NaN,0.05,0.0,0.24,0.29,1.18,1.00,24,14,156,73,156,73,76,25,1.00,0.00,Arsenal,Kai Havertz,1,23-24,2023-08-14 21:41:02.445217
2,NaN,NaN,444145,0,0,0,0,0,3,3.3,3.3,5,Gabriel,5.0,12,False,NaN,NaN,80,444145.jpg,5.0,Martinelli Silva,15.1,False,NaN,a,1,3,5,41226,41226,17905,17905,0.6,0.6,Martinelli,85,0,1,0,1,0,0,0,0,0,0,0,23,25.4,42.5,8.0,7.6,1,0.00,0.22,0.22,1.16,46,16,11,7,82,48,27,15,4.0,NaN,3.0,NaN,3.0,NaN,0.00,0.0,0.23,0.23,1.23,1.06,16,9,51,16,51,16,29,8,1.06,0.00,Arsenal,Gabriel Martinelli Silva,1,23-24,2023-08-14 21:41:02.445217
3,NaN,NaN,205533,0,0,0,0,0,4,2.4,2.4,8,Eddie,8.0,13,False,NaN,NaN,55,205533.jpg,8.0,Nketiah,4.0,False,NaN,a,1,3,8,68097,68097,8998,8998,1.5,1.5,Nketiah,72,1,0,1,0,0,0,0,0,0,0,2,26,30.2,5.9,36.0,7.2,1,0.21,0.01,0.22,0.30,35,11,110,10,16,9,31,11,NaN,NaN,NaN,NaN,NaN,NaN,0.26,0.0,0.01,0.27,0.38,0.00,121,33,17,5,17,5,88,16,1.25,1.25,Arsenal,Eddie Nketiah,1,23-24,2023-08-14 21:41:02.445217
4,NaN,NaN,184029,0,0,0,0,0,3,3.4,3.4,2,Martin,2.0,14,False,NaN,NaN,85,184029.jpg,2.0,Ødegaard,23.6,False,NaN,a,1,3,2,31704,31704,64463,64463,0.2,0.2,Ødegaard,90,0,0,0,1,0,0,0,0,0,0,0,11,12.0,23.4,14.0,4.9,1,0.11,0.09,0.20,1.18,110,40,39,27,60,36,68,38,3.0,NaN,1.0,NaN,NaN,NaN,0.11,0.0,0.09,0.20,1.18,1.00,8,6,158,74,158,74,14,6,1.00,0.00,Arsenal,Martin Ødegaard,1,23-24,2023-08-14 21:41:02.445217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,NaN,NaN,247632,0,0,0,0,0,3,2.1,1.6,1,Pedro,1.0,567,False,NaN,NaN,55,247632.jpg,1.0,Lomba Neto,0.2,False,NaN,a,20,39,1,1999,1999,886,886,0.2,0.2,Neto,90,0,0,0,1,0,0,0,1,0,0,0,19,20.2,65.4,46.0,13.2,1,0.79,0.35,1.14,1.

In [ ]:
# running team data from past seasons
filepath = Path('../data/modeling/team_data.csv')
team_data = pd.read_csv(filepath, index_col=0)
display(team_data.head())
display(team_data.tail())
display(team_data.shape)

In [ ]:
# fpl fixtures data from this season
filepath = Path('../data/fixtures/fpl_fixtures.csv')
fpl_fixtures = pd.read_csv(filepath, index_col=0)
display(fpl_fixtures.head())
display(fpl_fixtures.shape)

In [ ]:
# fbref fixtures data from this season
filepath = Path('../data/fixtures/fbref_fixtures.csv')
fbref_fixtures = pd.read_csv(filepath, index_col=0)
display(fbref_fixtures.head())
display(fbref_fixtures.shape)

# Process FPL data

In [ ]:
# find how many minutes a player played on a given gameweek
fpl_df['gameweek_minutes'] = fpl_df.groupby(['first_name', 'second_name', 'season'])['minutes'].diff()
# fill na caused at the start of each season by taking diff (but don't fill for season 22-23 where early season data is missing)
fpl_df['gameweek_minutes'] = fpl_df.apply(lambda x: my_fill_na(x, 'gameweek_minutes', 'minutes'), axis=1)
print('Number of rows with zero minutes played in a gameweek:')
display(fpl_df[fpl_df.gameweek_minutes==0].shape[0])
print('Number of rows with over 90 minutes played in a gameweek:')
display(fpl_df[fpl_df.gameweek_minutes>90].shape[0])

In [ ]:
# check does the latest season have any problem data (ok if '23-24' does not appear here)
display(fpl_df.loc[fpl_df.gameweek_minutes>90, 'season'].unique())
display(fpl_df.loc[fpl_df.gameweek_minutes==0, 'season'].unique())

In [ ]:
# drop rows with 0 minutes or more than 90 minutes
fpl_df = fpl_df[(fpl_df.gameweek_minutes>0) & (fpl_df.gameweek_minutes<=90)].reset_index(drop=True)
display(fpl_df.head())
display(fpl_df.shape)

### Add xG data to FPL fixtures data

In [ ]:
# map fixtures team names to fpl team names

for i in range(len(fixtures)):
    # map fbref team names to fpl team names
    team_name_dict = dict(zip(np.sort(fixtures[i].home_team.unique()), np.sort(fpl_df.loc[fpl_df.season==seasons[i], 'team_name'].unique())))
    display(team_name_dict)

    fixtures[i]['home_team'] = fixtures[i]['home_team'].apply(lambda x: team_name_dict[x])
    fixtures[i]['away_team'] = fixtures[i]['away_team'].apply(lambda x: team_name_dict[x])
    display(fixtures[i].head())

In [ ]:
dict(zip(np.sort(fpl_fixtures.home_team.unique()), np.sort(fpl_df.loc[fpl_df.season=='23-24', 'team_name'].unique())))

In [ ]:
fpl_fixtures.home_team.unique()